# Exercise 7: NetCDF4

## Aim: Introduce the netCDF4 library in Python to read and create NetCDF4 Files.

### Issues covered:
- Importing netCDF4
- Reading a NetCDF file as a Dataset instance
- Accessing dimensions, variables and attributes
- Defining dimensions, variables and attributes
- Writing a NetCDF file as a Dataset

## Creating/opening/closing netCDF files

- Import the `netCDF4` library
- Let's create a new NetCDF file called "test.nc" in all? mode ('a') with the NETCDF4 format. This mode will allow us to edit the dataset later.
- Inspect the new file to see what its `data_model` is.

In [1]:
# Step 1: Import netCDF4 library
import netCDF4
# Step 2: Create the new file
new_file = netCDF4.Dataset("test.nc", "a", format="NETCDF4")
# Step 3: Check the new file out
new_file.data_model

## Groups, dimensions, variables and attributes

### Groups

- Groups act as containers for variables, dimensions and attributes. Let's add a group to the dataset we just made called "forecasts".
- List the groups of your dataset using `.groups`
- Create a new group within forecasts called `model1` then print the groups to see your new group.
- What happens if you do `group3 = new_file.createGroup("/analyses/model2")`?
- What happens if you do `group4 = new_file.createGroup("analyses")`?

In [4]:
# Step 1: 
group1 = new_file.createGroup("forecasts")
# Step 2: 
new_file.groups
# Step 3: 
group2 = new_file.createGroup("/forecasts/model1")
new_file.groups
# Step 4: 
# It creates the 'analyses' group then adds the 'model2' group to it.
group3 = new_file.createGroup("/analyses/model2")
new_file.groups
# Step 5: 
# Nothing - it returns the existing group.
group4 = new_file.createGroup("analyses")
new_file.groups

### Dimensions

- Create some dimensions for the `new_file` dataset:
    - time dimension with unlimited size
    - level dimension with unlimited size
    - lat dimension with unlimited size
    - lon dimension with unlimited size
- Print out the dimensions you just created.
- Check the length of the latitude dimension to make sure it is 10.
- Check that the level dimension is unlimited.
- Let's take a look at an overview using 
```
for dim in new_file.dimensions.values():
    print(dim)
```

In [9]:
# Step 1:
time = new_file.createDimension('time', None)
level = new_file.createDimension('level', None)
lat = new_file.createDimension('lat', None)
lon = new_file.createDimension('lon', None)
# Step 2:
new_file.dimensions
# Step 3: 
print(len(lat))
# Step 4: 
print(level.isunlimited())
# Step 5: 
for dim in new_file.dimensions.values():
    print(dim)

### Variables

Remember that the data types are as follows:
- `f4`: 32-bit floting point 
- `f8`: 64-bit floating point 
- `i4`: 32-bit signed integer 
- `i2`: 16-bit signed integer
- `i8`: 64-bit unsigned integer
- `i1`: 8-bit signed integer
- `u1`: 8-bit unsigned integer
- `u2`: 16-bit unsigned integer
- `u4`: 32-bit unsigned integer
- `u8`: 64-bit unsigned integer
- `S1`: single-character string

- Create a scalar variable called `times` with the type set to `f8`.
- Create a scalar variable called `levels` but this time set the type to `np.float64`. (You'll need to import nump as np)
- Print out the variables using `new_file.variables`. What do you notice about the types?
- Create a variable in the model2 group we made earlier called `temp`, with the float64 type and this time give it dimensions: (`time`, `level`, `lat`, `lon`). Print it out.
- Create two values: 
    - longitudes with the name "lon", type float64 and dimension lon
    - latitudes with the name "lat", type float64 and dimension lat

In [14]:
# Step 1:
times = new_file.createVariable('times', 'f8')
# Step 2: 
import numpy as np
levels = new_file.createVariable('levels', np.float64)
# Step 3:
# The types are the same - both float64. Sometimes people will use np.float64 as it is more clear than f8. 
print(new_file.variables)
# Step 4: 
temp = new_file.createVariable("/analyses/model2/temp", np.float64, ("time", "level", "lat", "lon",))
print(new_file.variables)
# Step 5: 
longitudes = new_file.createVariable("lon", np.float64, ("lon",))
latitudes = new_file.createVariable("lat", np.float64, ("lat",))

### Attributes

- Let's create a global attribute. Create an attribute on the new_file object called `.description` with the value `This is a test description.`.
- Let's create a variable attribute. Create an attribute on the `times` variable called `units` and put `hours`.
- Take a look at the attrs on `new_file` using `new_file.ncattrs()`. What does this show?
- To get the name AND description, use the following loop:
```
for name in new_file.ncattrs():
    print(name, ":", getattr(new_file, name))
```
- There is an easier way of doing this - using `new_file.__dict__`. Try it out!

In [19]:
# Step 1: 
new_file.description = "This is a test description."
# Step 2:
times.units = "hours"
# Step 3: 
#This just shows the name of the global attrs.
new_file.ncattrs()
# Step 4:
for name in new_file.ncattrs():
    print(name, ":", getattr(new_file, name))
# Step 5:
new_file.__dict__

## Writing data to and receiving data from netCDF variables

- Create an array to populate the lats with using `lats = np.arange(-100, 100, 2)` and an array to populate the lons with using `lons = np.arange(-200, 200, 2)`.
- Print out latitudes and longitudes to see what it looks like before we populate these variables.
- Populate the two variables with our data using `latitudes[:] = lats` and the same for longitudes.
- Print the data out and take a look.

In [24]:
# Step 1: 
lats = np.arange(-90, 91, 5)
lons = np.arange(-180, 180, 5)
# Step 2: 
print(latitudes)
print(longitudes)
# Step 3: 
latitudes[:] = lats
longitudes[:] = lons
# Step 4: 
print("latitudes =\n{}".format(latitudes[:]))
print("longitudes =\n{}".format(longitudes[:]))

- Extend new_file to include dimensions for `time` and `pressure` where time is an unlimited dimension.
- Define a 4D variable `temperature` with dimensions (time, pressure, latitude, longitude)
- Generate random temperature data for a subset of time and pressure values and assign it to `temperature`. Use dimensions (10, 3, 37, 73) where `time` ranges from 0 to 9, `pressure` has three levels (850, 500 and 200 hPa).
- After assigning the data, print the shape of the `temperature` variable.

In [28]:
import numpy as np

new_file.createDimension("pressure", 10)

temperature = new_file.createVariable("temperature", "f4", ("time", "pressure", "lat", "lon",))

nlats = len(new_file.dimensions["lat"])
nlons = len(new_file.dimensions["lon"])

temperature[0:10, 0:3, :, :] = np.random.uniform(size=(10, 3, nlats, nlons))

print("temp shape after adding data = {}".format(temperature.shape))

temp shape after adding data = (10, 10, 37, 72)


- Define the `pressure` variable with values [1000, 850, 700, 500, 300, 250, 200, 150, 100, 50].
- Populate the `pressure` variable in the netCDF dataset.
- Use fancy indexing to slice the temoerature variable: select times 0, 2 and 4. Use pressure levels [850, 500, 200] and select only positive latitudes and longitudes.
- Print the shape of the resulting subset array.

In [29]:
pressure = new_file.createVariable("pressure", "f4", ("pressure",))

pressure[:] = [1000., 850., 700., 500., 300., 250., 200., 150., 100., 50.]

temperature = new_file.variables["temperature"]
latitudes = new_file.variables["lat"][:]
longitudes = new_file.variables["lon"][:]

tempdat = temperature[::2, [1, 3, 6], latitudes > 0, longitudes > 0]
print("shape of fancy temp slice = {}".format(tempdat.shape))

shape of fancy temp slice = (5, 3, 18, 35)


## Time-coordinates

Most metadata standards specify that time should be measured relative to a fixed date with units such as `hours since YY-MM-DD hh:mm:ss`. We can convert values to and from calendar dates using `num2date` and `date2num` from the `cftime` library. Two other helpful functions are `datetime` and `timedelta` from the `datetime` library.

- Let's generate a list of data and time values: create a list called `dates` containing date and time values, starting from January 1st 2022, and incrementing by 6 hours for a total of 5 entries. 
- Use `date2num` to convert your list of dates to numeric values using: `units="hours since 2022-01-01 00:00:00"` amd `calendar="gregorian"`. Store these in an array called `times`.
- Print the numeric times values to confirm the numeric representation.
- Use `num2date` to convert times back to datetime objects using the same units and calendar. Store these in a list called `converted_dates`
- Print the converted dates to verify they match the original dates list. 

In [33]:
from datetime import datetime, timedelta
from cftime import num2date, date2num

# Step 1: Generate dates list
dates = [datetime(2022, 1, 1) + n * timedelta(hours=6) for n in range(5)]
print("Original dates:", dates)

# Step 2: Convert dates to numeric time values
units = "hours since 2022-01-01 00:00:00"
calendar = "gregorian"
times = date2num(dates, units=units, calendar=calendar)

# Step 3: Print numeric time values
print("Numeric time values (in units '{}'):\n{}".format(units, times))

# Step 4: Convert numeric time values back to calendar dates
converted_dates = num2date(times, units=units, calendar=calendar)

# Step 5: Print converted dates
print("Dates corresponding to numeric time values:\n", converted_dates)

Original dates: [datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 1, 1, 6, 0), datetime.datetime(2022, 1, 1, 12, 0), datetime.datetime(2022, 1, 1, 18, 0), datetime.datetime(2022, 1, 2, 0, 0)]
Numeric time values (in units 'hours since 2022-01-01 00:00:00'):
[ 0  6 12 18 24]
Dates corresponding to numeric time values:
 [cftime.DatetimeGregorian(2022, 1, 1, 0, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2022, 1, 1, 6, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2022, 1, 1, 12, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2022, 1, 1, 18, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2022, 1, 2, 0, 0, 0, 0, has_year_zero=False)]


## Multi-file datasets

Let's create multiple netCDF files with a shared variable and unlimited dimension, and use `MFDataset` to read the aggregated data as if it were contained in a single file.
- Create 5 netCDF files named `datafile0.nc` through to `datafile4.nc`. Each file should contain:
    - A single unlimited dimension named `time`.
    - A variable named `temperature` with 10 integer values ranging from `file_index * 10` to `(file_index+1) * 10 - 1`.
    - Ensure each file is saved in the `NETCDF4_CLASSIC` format.
- Using `MFDataset` read all the `temperature` data from the 5 files at once by specifying a wildcard string `datafile*.nc`.
- Print the aggregated `temperature` values to verify that they span from 0 to 49.

In [34]:
from netCDF4 import Dataset, MFDataset
import numpy as np

# Step 1: Create multiple netCDF files with a shared unlimited dimension and variable
for i in range(5):
    with Dataset(f"datafile{i}.nc", "w", format="NETCDF4_CLASSIC") as f:
        # Create an unlimited dimension
        f.createDimension("time", None)
        # Create a variable associated with the 'time' dimension
        temp_var = f.createVariable("temperature", "i4", ("time",))
        # Populate 'temperature' with a unique range of values for each file
        temp_var[:] = np.arange(i * 10, (i+1) * 10)

# Step 2: Use MFDataset to read all files at once
try:
    #Read and aggregate all data across all files
    f = MFDataset("datafile*.nc")
    temperature_data = f.variables["temperature"][:]
    # Print the aggregated data
    print(temperature_data)
finally:
    # Close the MFDataset object
    f.close()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]


## Compression of variables

Let's explore various compression options available in netCDF. 
- Create a sample NetCDF file: define a dataset with dimensions (`time`, `level`, `lat`, `lon`)
- Generate random temperature data.
- First, create the temperature variable without compression and observe the file size.
- Then, enable zlib compression and observe the change in file size.
- Finally, add quantization with `least_significant_digit` or `signigicant_digits` and check the file size again.

In [37]:
import os

# Step 1: Create a random dataset 
file_path = "temperature_data.nc"
time_dim, level_dim, lat_dim, lon_dim = 10, 5, 50, 100
data = np.random.rand(time_dim, level_dim, lat_dim, lon_dim) * 30 + 273.15

# Step 2: Define a function to create netCDF file with specified compression settings
def create_netcdf(file_path, compression=None, least_significant_digit=None, significant_digits=None):
    with Dataset(file_path, 'w', format="NETCDF4") as rootgrp:
        # Create dimensions
        rootgrp.createDimension("time", time_dim)
        rootgrp.createDimension("level", level_dim)
        rootgrp.createDimension("lat", lat_dim)
        rootgrp.createDimension("lon", lon_dim)
        # Define variable with compression settings
        temp = rootgrp.createVariable("temp", "f4", ("time", "level", "lat", "lon"), compression=compression, least_significant_digit=least_significant_digit, significant_digits=significant_digits)
        #Assign data to the variable
        temp[:] = data
    # Check and print file size
    print(f"File size with compression={compression}, "
          f"least_significant_digit={least_significant_digit}, "
          f"significant_digits={significant_digits}: {os.path.getsize(file_path) / 1024:.2f} kB")

# Step 3: Test different compression settings
# 3.1 No compression
create_netcdf("temperature_data_nocompress.nc")

# 3.2 Compression with zlib only
create_netcdf("temperature_data_zlib.nc", compression='zlib')

# 3.3 Compression with zlib and least significant digit quantization
create_netcdf("temperature_data_zlib_lsd.nc", compression='zlib', least_significant_digit=3)

# 3.4 Compression with zlib and significant digits quantization
create_netcdf("temperature_data_zlib_sig.nc", compression='zlib', significant_digits=4)

File size with compression=None, least_significant_digit=None, significant_digits=None: 983.31 kB
File size with compression=zlib, least_significant_digit=None, significant_digits=None: 639.48 kB
File size with compression=zlib, least_significant_digit=3, significant_digits=None: 505.94 kB
File size with compression=zlib, least_significant_digit=None, significant_digits=4: 396.39 kB


## Compound data types

Let's work with compound data types and structured arrays.
- Create a netCDF file called `vectors.nc` in write mode.
- Define a compound data type that represents a 3D vector. Each vector should have 3 components:
    - `x`: a `float33` representing the x-coordinate
    - `y`: a `float32` representing the y-coordinate
    - `z`: a `float32` representing the z-coordinate
- Create a dimension named `num_vectors` to store an unlimited number of vectors
- Create a variable in the file using the compound data type from step 2, with the dimension from step 3.
- Generate a numpy structured arrat with 5 sample 3D vectors:
    - Each vector should have random values for `x`, `y` and `z` components.
    - Store these in the structured array and write them to the netCDF variable.
- Close the file and then reopen it in read mode.
- Read the data back into a new numpy structured array and print each vector.  

In [38]:
# Step 1: Create a netCDF file in write mode.
f = Dataset("vectors.nc", "w", format="NETCDF4")

# Step 2: Define a compound data type for a 3D vector with x,y,z as float32 fields
vector_dtype = np.dtype([("x", np.float32), ("y", np.float32), ("z", np.float32)])
vector_t = f.createCompoundType(vector_dtype, "vector3D")

# Step 3: Create a dimension for storing an unlimited number of vectors
num_vectors = f.createDimension("num_vectors", None)

# Step 4: Create a variable using the compound data type and the dimension
vector_var = f.createVariable("vector_data", vector_t, ("num_vectors",))

# Step 5: Generate a numpy structured array with 5 random 3D vectors
num_samples = 5
data = np.empty(num_samples, dtype=vector_dtype)
data["x"] = np.random.rand(num_samples)
data["y"] = np.random.rand(num_samples)
data["z"] = np.random.rand(num_samples)

# Write the structured array to the netCDF variable
vector_var[:] = data

# Close the file
f.close()

#Step 6: Reopen the file in read-mode
f = Dataset("vectors.nc", "r")
vector_var = f.variables["vector_data"]

# Step 7: Read the data back into a new structured array and print each vector
data_in = vector_var[:]
for i, vec in enumerate(data_in):
    print(f"Vector {i}: (x: {vec['x']:.2f}, y: {vec['y']:.2f}, z: {vec['z']:.2f})")

# Close the file
f.close()

Vector 0: (x: 0.42, y: 0.18, z: 0.29)
Vector 1: (x: 0.24, y: 0.42, z: 0.55)
Vector 2: (x: 0.34, y: 0.50, z: 0.46)
Vector 3: (x: 0.42, y: 0.16, z: 0.68)
Vector 4: (x: 0.71, y: 0.72, z: 0.03)


## Variable-length data types

## Enum data type

## Extension